In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
import tensorflow as tf
#import keras_core as keras
import keras_nlp
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

import os
os.environ["KERAS_BACKEND"] = "tensorflow" 

import keras
import keras_nlp

import tensorflow as tf

import json
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

import plotly.graph_objs as go
import plotly.express as px

2024-05-16 22:09:42.229747: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-16 22:09:42.229831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-16 22:09:42.358851: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
print("TensorFlow version:", tf.__version__)
print("KerasNLP version:", keras_nlp.__version__)

TensorFlow version: 2.15.0
KerasNLP version: 0.9.3


In [3]:
model_type="distil_bert_base_en_uncased"

In [4]:
reviews = pd.read_json('/kaggle/input/imdb-spoiler-dataset/IMDB_reviews.json', lines=True)
def replace_string_binary(x):
    if x:
        return 1
    return 0

reviews['is_spoiler'] = reviews['is_spoiler'].apply(lambda x: replace_string_binary(x))
reviews

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,1,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,1,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,1,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,1,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,1,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,0,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,0,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,0,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,0,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


In [6]:
# Split data into training and test sets (80% train, 20% test)
train_data, test_data = train_test_split(reviews, test_size=0.1, random_state=42)

# Define a function to parse the DataFrame
def parse_dataframe(df):
    # Return features (text review) and labels (sentiment label)
    return df["review_text"].values, df["is_spoiler"].values

# Convert training and test data to TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices(parse_dataframe(train_data)).batch(32)
test_dataset = tf.data.Dataset.from_tensor_slices(parse_dataframe(test_data)).batch(32)

In [7]:
tokenizer = keras_nlp.models.DistilBertTokenizer.from_preset(model_type)
backbone = keras_nlp.models.DistilBertBackbone.from_preset(model_type)

Attaching 'tokenizer.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.txt' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/distil_bert/keras/distil_bert_base_en_uncased/2' to your Kaggle notebook...


In [8]:
packer = keras_nlp.layers.MultiSegmentPacker(
    start_value=tokenizer.cls_token_id,
    end_value=tokenizer.sep_token_id,
    pad_value=tokenizer.pad_token_id,
    sequence_length=500
)

def preprocess(x, y):
    token_ids, _ = packer(tokenizer(x))
    x = {
        "token_ids": token_ids,
        "padding_mask": token_ids != tokenizer.pad_token_id,
    }
    return x, y

train_ds = train_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = test_dataset.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)

In [9]:
from tensorflow.keras import callbacks

early_stopping = callbacks.EarlyStopping(
    min_delta=0.001, # minimium amount of change to count as an improvement
    patience=3, # how many epochs to wait before stopping
    restore_best_weights=True,
)

checkpoint = keras.callbacks.ModelCheckpoint(
    "./spoiler.keras",
    monitor='val_accuracy',
    verbose=0,
    save_best_only=True,
    mode='auto'
)

rlr = keras.callbacks.ReduceLROnPlateau(
        monitor='val_accuracy', 
        factor=1/3, 
        patience=3, 
        min_lr=1e-6
)

callbacks = [early_stopping, checkpoint, rlr]

In [ ]:
inputs = backbone.input
outputs = backbone(inputs)[:, 0, :]
#outputs = keras.layers.Dense(2028,activation='relu')(outputs)
outputs = keras.layers.Dense(1024,activation='relu')(outputs)
outputs = keras.layers.Dense(512,activation='relu')(outputs)
outputs = keras.layers.Dense(256, activation='relu')(outputs)
outputs = keras.layers.Dense(128,activation='relu')(outputs)

outputs = keras.layers.Dropout(0.1)(outputs)

outputs = keras.layers.Dense(2)(outputs)
    
model = keras.Model(inputs, outputs)
model.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=keras.optimizers.Adam(1e-5),
     metrics= ["accuracy"],
    jit_compile=True,
    
)

history=model.fit(
    train_ds, 
    validation_data=val_ds,
    epochs=2, 
    callbacks=callbacks
)

Epoch 1/2


I0000 00:00:1715897543.567237     116 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1715897543.613625     116 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


  786/16142 ━━━━━━━━━━━━━━━━━━━━ 3:07:00 731ms/step - accuracy: 0.7469 - loss: 0.5255

## HuggingFace Transformers

In [2]:
!pip install peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 4.6 MB/s eta 0:00:00a 0:00:01


In [13]:
from transformers import AutoTokenizer, DataCollatorWithPadding
import pandas as pd

reviews = pd.read_json('/kaggle/input/imdb-spoiler-dataset/IMDB_reviews.json', lines=True)
def replace_string_binary(x):
    if x:
        return 1
    return 0

reviews['label'] = reviews['is_spoiler'].apply(lambda x: replace_string_binary(x))
df = reviews[['review_text', 'label']]
df.head()

,review_text,label
0,"In its Oscar year, Shawshank Redemption (writt...",1
1,The Shawshank Redemption is without a doubt on...,1
2,I believe that this film is the best story eve...,1
3,"**Yes, there are SPOILERS here**This film has ...",1
4,At the heart of this extraordinary movie is a ...,1


In [14]:
from datasets import Dataset

hf_dataset = Dataset.from_pandas(df)
hf_dataset

Dataset({
    features: ['review_text', 'label'],
    num_rows: 573913
})

In [16]:
checkpoint = "google-bert/bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_fn(examples):
    return tokenizer(examples['review_text'], truncation=True)

tokenized_datasets = hf_dataset.map(tokenize_fn, batched=True)
data_collator = DataCollatorWithPadding(tokenizer)

Map:   0%|          | 0/573913 [00:00<?, ? examples/s]

In [17]:
tokenized_datasets

Dataset({
    features: ['review_text', 'label', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 573913
})

In [18]:
from transformers import AutoModelForSequenceClassification
from peft import get_peft_model, LoraConfig, TaskType
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
peft_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    inference_mode=False,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)
model.print_trainable_parameters()

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at google-bert/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


trainable params: 296,450 || all params: 108,608,260 || trainable%: 0.2730


In [19]:
split_datasets = tokenized_datasets.train_test_split(test_size=0.2)

In [20]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    "test-trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    learning_rate=1e-5,
    weight_decay=0.01
)

In [21]:
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset=split_datasets["train"],
    eval_dataset=split_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Step,Training Loss



KeyboardInterrupt

